In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [7]:
# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비


In [3]:
cifar10 = tf.keras.datasets.cifar10 #32x32x3 -> 10개의 클래스 구별

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

## Keras API 모델 학습

In [9]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
1563/1563 [==============================] - 21s 14ms/step - loss: 1.4417 - accuracy: 0.4742 - val_loss: 1.1293 - val_accuracy: 0.5934
Epoch 2/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.9755 - accuracy: 0.6520 - val_loss: 0.8884 - val_accuracy: 0.6907
Epoch 3/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.7576 - accuracy: 0.7310 - val_loss: 0.8103 - val_accuracy: 0.7243
Epoch 4/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.5882 - accuracy: 0.7942 - val_loss: 0.8068 - val_accuracy: 0.7310
Epoch 5/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.4220 - accuracy: 0.8511 - val_loss: 0.9170 - val_accuracy: 0.7301
Epoch 6/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.2741 - accuracy: 0.9040 - val_loss: 1.1330 - val_accuracy: 0.7317
Epoch 7/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.1874 - accuracy: 0.9339 - val_loss: 1.2960 -

# 모델변경

In [4]:
class ConvLNReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvLNReluBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias = False)
        self.ln = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.ln(x)
        return tf.nn.relu(x)
    

# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = ConvLNReluBlock(16, (3, 3)) 
        self.conv1_2 = ConvLNReluBlock(16, (3, 3)) 
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = ConvLNReluBlock(32, (3, 3)) 
        self.conv2_2 = ConvLNReluBlock(32, (3, 3)) 
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = ConvLNReluBlock(64, (3, 3)) 
        self.conv3_2 = ConvLNReluBlock(64, (3, 3)) 

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', 
                                            kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax',
                                           kernel_regularizer = tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [ ]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
 429/1563 [=======>......................] - ETA: 1:07 - loss: 3.7554 - accuracy: 0.2416